In [ ]:
## Multi Layer Neural network

In [ ]:
## in previous Notebook we have seen used single layer perceptron neural network and  Bi gram stats model both of them cannot capture the context better, they dont sound the names better
# so in this notebook we use the multilayer perceptron which are useful to generate the meaning full name

# so the problem with bi-gram model we face is we are able to capture the context of 2 characters at a time.(that gives us 27 character * 27 characters = 729 possibilities combination comes in context)
# if we have the 3 characters as context  we would get (27*27*27 =19,623 combinations context) generating count vector normalizing and everything becomes difficult now.
# the modeling approach we gonna follow  is from this research paper bengio et al. 2003 (https://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf)




In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
words = open('names.txt', 'r').read().splitlines()
words[:5]

['emma', 'olivia', 'ava', 'isabella', 'sophia']

In [7]:
chars = sorted(list(set(''.join(words))))
stoi = {
    s: i+1 for i,s in enumerate(chars)
}
stoi['.'] = 0
itos = { i:s for s,i in stoi.items() }


{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


## Creating dataset for Neural Networks from Text File to Input Tensor and Output Tensor

In [37]:
## data set for neural network

block_size = 3 #Context_length: how many characters do we take to predict the next one?
X,Y =[],[] # x are inputs and Y are labels

for w in words[:1]:

    print(w)
    context = [0] * block_size

    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)

        print("input data : ",''.join(itos[i] for i in context), ", Output :", '=', itos[ix])
        print('context :', context)
        context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)


print('\n')
print("input tensor :", X)

print("output tensor :", Y)

emma
input data :  ... , Output : = e
context : [0, 0, 0]
input data :  ..e , Output : = m
context : [0, 0, 5]
input data :  .em , Output : = m
context : [0, 5, 13]
input data :  emm , Output : = a
context : [5, 13, 13]
input data :  mma , Output : = .
context : [13, 13, 1]


input tensor : tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        [ 5, 13, 13],
        [13, 13,  1]])
output tensor : tensor([ 5, 13, 13,  1,  0])


In [ ]:
"""
Above output explanation

you can notes how the input data is store in the input tensor it is simple

input data :  ... , Output : = e
context : [0, 0, 0]

when we get input '...' we have the o/p as 'e'
similary i/p = '..e' => o/p = 'm'

-> we are trying to take context of 3 character at same time and trying to predict the o/p of what character come next and run in the loop

1)so we are able to use above block and change the parameters like block size to get different context
2) if we consider all the input and one cblock size we are able to get the whole i/p and o/p data



"""

# Block size = 5

In [38]:
block_size = 5 #Context_length: how many characters do we take to predict the next one?
X,Y =[],[] # x are inputs and Y are labels

for w in words[:1]:

    print(w)
    context = [0] * block_size

    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)

        print("input data : ",''.join(itos[i] for i in context), ", Output :", '=', itos[ix])
        print('context :', context)
        context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)


print('\n')
print("input tensor :", X)

print("output tensor :", Y)

emma
input data :  ..... , Output : = e
context : [0, 0, 0, 0, 0]
input data :  ....e , Output : = m
context : [0, 0, 0, 0, 5]
input data :  ...em , Output : = m
context : [0, 0, 0, 5, 13]
input data :  ..emm , Output : = a
context : [0, 0, 5, 13, 13]
input data :  .emma , Output : = .
context : [0, 5, 13, 13, 1]


input tensor : tensor([[ 0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  5],
        [ 0,  0,  0,  5, 13],
        [ 0,  0,  5, 13, 13],
        [ 0,  5, 13, 13,  1]])
output tensor : tensor([ 5, 13, 13,  1,  0])


### above we are trying to take 5 character as input and predicting what comes next as o/p

In [49]:
block_size = 3 #Context_length: how many characters do we take to predict the next one?
X,Y =[],[] # x are inputs and Y are labels

for w in words:

    #print(w)
    context = [0] * block_size

    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)

        #print("input data : ",''.join(itos[i] for i in context), ", Output :", '=', itos[ix])
        #print('context :', context)
        context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)
X

tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        ...,
        [26, 26, 25],
        [26, 25, 26],
        [25, 26, 24]])

In [50]:
import pandas as pd
pd.DataFrame(Y)

,0
0,5
1,13
2,13
3,1
4,0
...,...
228141,26
228142,25
228143,26
228144,24


In [51]:
"""
1. now the Test data with the block_size/context = 3 is ready and o/p is also ready fo the model to get trained on.
2.
"""

'\n1. now the Test data with the block_size/context = 3 is ready and o/p is also ready fo rthe model to get trianned on\n'

In [53]:
(X.shape, X.dtype), (Y.shape, Y.dtype)

((torch.Size([228146, 3]), torch.int64), (torch.Size([228146]), torch.int64))

In [ ]:
## so our tensor matrix x has 228146 rows and 3 columns only both of there dtype is integer

## In paper they have 17,000 words and they embedded in 30 dimensional

## in our case we have only 27 possible characters so let's consider 2- dimensional space so we have 27 charcter with 2 dimenisonal space(27*2) tensor

In [60]:
# let embedded the 2 random numbers with 27*2

C = torch.randn((27,2)) # the weight Matrix is C it is represented as C because of research paper
C

tensor([[-0.5027,  0.8715],
        [ 1.6704,  0.8234],
        [-0.5630, -0.2439],
        [ 0.2107, -1.3640],
        [-0.7531,  0.3911],
        [ 0.0965,  0.0613],
        [ 0.6137, -2.2380],
        [ 0.1893,  0.3401],
        [-0.9191, -1.2767],
        [ 0.9177, -0.0755],
        [ 1.6452,  2.2990],
        [-1.1870,  1.7304],
        [-1.0177, -0.5633],
        [ 1.0176,  0.5821],
        [-0.5160, -1.0842],
        [ 1.7583,  0.1848],
        [-2.1371,  0.7803],
        [-1.5838,  0.6372],
        [-0.1884, -1.4087],
        [-0.3574, -0.2132],
        [-0.5377,  1.0338],
        [ 0.6379,  0.3611],
        [ 0.2035, -0.4935],
        [ 0.1628, -0.7074],
        [-1.0163,  0.7758],
        [-1.0250,  0.1586],
        [ 1.1187, -0.1005]])

In [57]:
C[5]

tensor([0.5309, 0.3936])

In [65]:
#Example

(F.one_hot(torch.tensor(5), num_classes=27).float()) @ C

tensor([0.0965, 0.0613])

In [ ]:
# if we multiply the weight vector(C) * index_vector/one_hot encoding vector of the input(X) = we get the Result from the first layer

In [66]:
C = torch.randn((27,2))
C ## c is noting buit one type of encoding

tensor([[-0.5978, -0.7147],
        [-0.5285,  0.5865],
        [-1.4994,  0.3971],
        [ 0.4928,  0.8248],
        [-0.2717,  1.1332],
        [ 0.7461,  0.4762],
        [ 1.1987,  0.3515],
        [-0.7940, -0.7886],
        [-0.5782, -0.0506],
        [-0.6811,  0.1497],
        [-0.1734, -0.1477],
        [ 0.5712,  0.0527],
        [-1.2045, -0.4160],
        [ 0.9480,  0.9380],
        [ 1.2135, -0.8686],
        [-0.8709, -0.4112],
        [-0.7289, -0.6578],
        [ 1.2973,  0.4326],
        [ 0.2009, -0.5566],
        [ 0.1841, -0.1151],
        [-0.1430,  0.0753],
        [-0.7440,  0.4990],
        [ 0.1834, -0.6401],
        [ 1.3906,  2.0530],
        [-1.4367, -0.9865],
        [-0.3766,  0.1291],
        [-0.0449,  1.5053]])

In [67]:
# we want to encode one number 5 we can simply pass this number to the Embedding tensor C and get the output i.e.,

C[5] #Give embedding of 5 in 2d Vector Space

tensor([0.7461, 0.4762])

In [69]:
# what if the embedding is like list

print(C[[2,3,4]]) # we got the embedding of 3 integer vector as below


tensor([[-1.4994,  0.3971],
        [ 0.4928,  0.8248],
        [-0.2717,  1.1332]])


In [71]:
# what if we give the tensor input to get embedding

print(C[torch.tensor([5,6,7,7,7,7,7,7,])]) # still we are able to get the embedding by giving tensor matrix as input

tensor([[ 0.7461,  0.4762],
        [ 1.1987,  0.3515],
        [-0.7940, -0.7886],
        [-0.7940, -0.7886],
        [-0.7940, -0.7886],
        [-0.7940, -0.7886],
        [-0.7940, -0.7886],
        [-0.7940, -0.7886]])


In [ ]:
"""
Full explanation of C

 C is the embedding vector  when you pass the single integer -> we get the embedding of that single integer
if we pass list to embedding vector c -> we goonna get embedding for that list passed
if we pass the tensor to embedding vector -. we gonna get th emebedding for that tensor


to get emebedding for anything we need to pass it to the C

"""

In [72]:
C[X]

tensor([[[-0.5978, -0.7147],
         [-0.5978, -0.7147],
         [-0.5978, -0.7147]],

        [[-0.5978, -0.7147],
         [-0.5978, -0.7147],
         [ 0.7461,  0.4762]],

        [[-0.5978, -0.7147],
         [ 0.7461,  0.4762],
         [ 0.9480,  0.9380]],

        ...,

        [[-0.0449,  1.5053],
         [-0.0449,  1.5053],
         [-0.3766,  0.1291]],

        [[-0.0449,  1.5053],
         [-0.3766,  0.1291],
         [-0.0449,  1.5053]],

        [[-0.3766,  0.1291],
         [-0.0449,  1.5053],
         [-1.4367, -0.9865]]])

In [73]:
C[X].shape

torch.Size([228146, 3, 2])

now our input is embeeded